<h1><span style="text-decoration: underline;"><span style="color: #000080;"><strong>Ballon Dor - First Step --> Crawling</strong></span></span></h1>

In [7]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from getpass import getpass
import time
pd.options.mode.chained_assignment = None

<h2><span style="text-decoration: underline; color: #000080;">Crawling From Wikipedia:</span></h2>
<ul>
<li><span style="color: #000080;">Taking tables between 2019-2007</span></li>
<li><span style="color: #000080;">Clear and handling the tables for relvant columns</span></li>
</ul>

In [ ]:
# קוד משיכת נתונים מוויקיפדיה

pd.options.mode.chained_assignment = None

wikipedia_urls = ["https://en.wikipedia.org/wiki/2018_Ballon_d%27Or","https://en.wikipedia.org/wiki/2007_Ballon_d%27Or",
           "https://en.wikipedia.org/wiki/2017_Ballon_d%27Or","https://en.wikipedia.org/wiki/2016_Ballon_d%27Or",
         "https://en.wikipedia.org/wiki/2008_Ballon_d%27Or","https://en.wikipedia.org/wiki/2009_Ballon_d%27Or"
             ,"https://en.wikipedia.org/wiki/2014_FIFA_Ballon_d%27Or","https://en.wikipedia.org/wiki/2015_FIFA_Ballon_d%27Or"
         ,"https://en.wikipedia.org/wiki/2013_FIFA_Ballon_d%27Or","https://en.wikipedia.org/wiki/2012_FIFA_Ballon_d%27Or",
          "https://en.wikipedia.org/wiki/2011_FIFA_Ballon_d%27Or","https://en.wikipedia.org/wiki/2010_FIFA_Ballon_d%27Or"]


years = ['2018-2019', '2017-2018', '2006-2007', '2016-2017','2015-2016','2007-2008','2008-2009','2013-2014','2014-2015','2012-2013',
        '2011-2012','2010-2011', '2009-2010']

data = pd.read_html("https://en.wikipedia.org/wiki/2019_Ballon_d%27Or")[1]

lens = []

prob = data["Points"].sum()
precent = []
for o in data['Points']:
    precent.append(o/prob*100)
    
data['precent'] = precent
data['Season'] = years[0]
j = 0
for i in wikipedia_urls:
    
    if j >= 8:
        j = j+1
        # מטפל בבעיה של הקישורים שבהם יש שתי טבלאות והמיקומים שלהן 0 ו 1
        df1 = pd.read_html(i)[0]
        df1['Season'] = years[j]
        data = pd.concat([data,df1], ignore_index=True, sort=False)
        
        if len(df1) == 3:
            # מכניס טבלה מספר 2 לדאטה 
            df2 = pd.read_html(i)[1]
            df2['Season'] = years[j]
            data = pd.concat([data,df2], ignore_index=True, sort=False)   
    else: 
        # מטפל בקישורים הראשונים כאשר בהם יש טבלה אחת 
        j = j+1
        df1 = pd.read_html(i)[1]
        df1['Season'] = years[j]
        
        if j == 1:
            df1['Points'][0] = df1['Points'][0].replace("[4]","")
        
        if j <= 6:
            # מבצע החלפה בין נקודות לאחוזים
            df1["Points"] = df1['Points'].astype(float)
            prob = df1["Points"].sum()
            precent = []
            
            for k in df1['Points']:
                precent.append(k/prob*100)   
            df1['precent'] = precent
            #df1['Season'] = years[j]
        data = pd.concat([data,df1], ignore_index=True, sort=False)        
        if len(df1) == 3:
            # מטפל בשנים 2014 -215 שבהם יש שתי טבלאות
            df2 = pd.read_html(i)[2]
            df2['Season'] = years[j]
            data = pd.concat([data,df2], ignore_index=True, sort=False)
            print(df2)




data    

In [ ]:


data['Rank'] = data['Rank'].fillna(data['Rhxxhy'])
data['Points'] = data['Points'].fillna(data['Votes'])
data['Player'] = data['Player'].fillna(data['Player[11]'])
data['Player'] = data['Player'].fillna(data['Player[2]'])
data['Player'] = data['Player'].fillna(data['Player[3]'])
data['Player'] = data['Player'].fillna(data['Player[3]'])
data['Percent'] = data['Percent'].fillna(data['precent'])
data = data.drop(['Player[11]','Player[2]','Player[3]','Rhxxhy','Votes','precent','Nationality','Points'],axis = 1)

In [ ]:
copy_of_players_remove_duplicates = data.copy()
list(set(copy_of_players_remove_duplicates)) # REMOVE DUPLICATES NAMES אין צורך בהם ללינקים

data.to_csv("player_list_wiki.csv")
data.to_csv("copy_of_players__wiki_remove_duplicates.csv")



In [ ]:
# player_list שימוש עיתידי לטבלה הסופית על מנת להוסיף את ההתסברות
#copy_of_players_remove_duplicates אין צורך בכפילויות של השחקנים בשביל להוציא לינק שלהם, לא יעיל

<h2><span style="text-decoration: underline; color: #000080;">Pulling Links of League use selenium:</span></h2>
<ul>
<li><span style="color: #000080;">Pulling link of all players from CSV</span></li>
<li><span style="color: #000080;">Saving the relvant links on new data</span></li>
</ul>

In [ ]:
chrome_options = Options()
chrome_options.add_argument("--kiosk")

file = pd.read_csv("copy_of_players__wiki_remove_duplicates.csv")
df = pd.DataFrame(file)
df = df.drop_duplicates(subset=['Player'], keep='first')
total_players = df['Player']
players_link_league = []

for player_name in total_players:
    driver = webdriver.Chrome("C:\Program Files (x86)\chromedriver.exe",chrome_options=chrome_options)
    driver.get("https://fbref.com/en/")
    search_box = driver.find_element_by_name("search")
    search_box.send_keys(player_name)
    search_button = driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[3]/form/div/div/div/div[1]/div[2]/div/div').click()
    players_link_league.append(driver.current_url)
    driver.quit()
    

    
df_links = pd.DataFrame({"Player":total_players,"link_of_player":players_link_league})
df_links
df_links.to_csv("Links_Of_Players_League.csv")

<h2><span style="text-decoration: underline; color: #000080;">Taking league details of players:</span></h2>
<ul>
<li><span style="color: #000080;">Build big data of all players on CSV</span></li>
<li><span style="color: #000080;">Save all players leauge details on new data</span></li>
</ul>

In [ ]:
df_league = pd.read_csv("Links_Of_Players_League.csv")
df_league

In [5]:
# פונקציה למשיכת נתונים מהליגה  

# פונקציה למשיכת נתונים מהליגה  

def Create_DataFrame_league(url,name):
    url1 = url
    responsel1 = requests.get(url1)
    soup2 = BeautifulSoup(responsel1.content, "html.parser")
    mtag1 = soup2.find("div", attrs = {"class":"table_container"})
    
    player,Season,age,squad,country,lg_rank,comp,mp,starts,minutes,goals,
    goals_per_game,assists,penalty_kicks_made,penalty_kicks_attempted,
    yellow_cards,red_cards,assists_per_game = [],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]

    j = 0

    for i in mtag1("th", attrs = {"class":"left"}):  # work
        if i.text == '':
            continue
        Season.append(i.text)
    for i in mtag1("td", attrs = {"class":"center"}): # work
        if i.text == '':
            continue
        age.append(i.text)
    for i in mtag1("td", attrs = {"class":"left"}): # work
        if j == 0:
            squad.append(i.text)
            j = j+1
        elif j == 1:
            country.append(i.text)
            j = j+1
        elif j == 2:
            comp.append(i.text)
            j = j+1
        else:
            j = 0

            j =0

    for i in mtag3("p"):
        j = j+1
        pos = i.text
        if pos[0:8] == "Position":
            break
    pos = pos[10:12]            
            
    for m in mtag1.findAll(id = "stats"):
        j = 0
        for i in m("td", attrs = {"class":"right"}):        
            if(i.text == None):
                continue
            
            if j == 0:
                lg_rank.append(i.text)
                j = j+1
            elif j == 1:
                mp.append(i.text)
                j = j+1    
            elif j == 2:
                starts.append(i.text)
                j = j+1
            elif j == 3:
                minutes.append(i.text)
                j = j+ 1 
            elif j == 4:
                j = j +1
            elif j == 5:
                goals.append(i.text)
                j= j+1
            elif j == 6:
                assists.append(i.text)
                j= j+1
            elif j == 7:
                penalty_kicks_made.append(i.text)
                j= j+1
            elif j == 8:
                penalty_kicks_attempted.append(i.text)
                j= j+1
            elif j == 9:
                yellow_cards.append(i.text)
                j = j+1 
            elif j == 10:
                red_cards.append(i.text)
                j = j+ 1 
            elif j == 11:
                goals_per_game.append(i.text)
                j= j+1
            elif j == 12:
                assists_per_game.append(i.text)
                j= j+1
            elif j < 12:
                break
                
    size = len(goals)
                
     
    for ijk in squad:
        if ijk == '':
            squad.remove('')
            
    for ijk in squad:
        if ijk == '':
            comp.remove('')
   
    for ijk in squad:
        if ijk == '':
            country.remove('') 
            
            
    while (len(squad) > size):
        squad.pop()

    while (len(Season) > size):
        Season.pop()
    
    while (len(comp) > size):
        comp.pop()    

    while (len(country) > size):
        country.pop()   
    
    while (len(age) > size):
        age.pop()                 

    for names in range (0,size):
        player.append(name)

    for names in range (0,size):
        position.append(pos) 
        
    df2 = pd.DataFrame({"Player":player,"Season":Season, "Age":age,"Position":position,"Squad":squad,"Country":country,"Comp":comp,"Lg_Rank":lg_rank,"Matches":mp,"Starts":starts,"Minutes":minutes,"Goals":goals,"Assists":assists,"penalty_kicks_made":penalty_kicks_made,"penalty_kicks_attempted":penalty_kicks_attempted,"yellow_cards":yellow_cards,"red_cards":red_cards,"goals_per_game":goals_per_game,"assists_per_game":assists_per_game })         
    return df2                  

In [ ]:

league_data = pd.DataFrame()
k = 0
for i in df_league['link_of_player']:
    name = df_league['Player'][k]
    df_assistent = Create_DataFrame_league(i,name)
    league_data = pd.concat([league_data,df_assistent], ignore_index=True, sort=False)
    k = k +1
league_data 

In [ ]:
league_data.to_csv("league_data.csv")

<h2><span style="text-decoration: underline; color: #000080;">Pulling Links of international use selenium:</span></h2>
<ul>
<li><span style="color: #000080;">Pulling link of all players from CSV</span></li>
<li><span style="color: #000080;">Saving the relvant links on new data</span></li>
</ul>

In [ ]:
chrome_options = Options()
chrome_options.add_argument("--kiosk")


file = pd.read_csv("copy_of_players__wiki_remove_duplicates.csv")
df = pd.DataFrame(file)
df = df.drop_duplicates(subset=['Player'], keep='first')
total_players = df['Player']
players_link_international = []
i = 1
for player_name in total_players:
    driver = webdriver.Chrome("C:\Program Files (x86)\chromedriver.exe",chrome_options=chrome_options)
    driver.get("https://fbref.com/en/")
    search_box = driver.find_element_by_name("search")
    time.sleep(3)
    search_box.send_keys(player_name)
    search_button = driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[3]/form/div/div/div/div[1]/div[2]/div/div').click()
    time.sleep(1)
    check_national = driver.find_element_by_xpath('/html/body/div[2]/div[6]/div[2]/div[4]')
    if check_national.text == 'International Cups':
        internationl_cups_button = driver.find_element_by_xpath('/html/body/div[2]/div[6]/div[2]/div[4]').click()
    else:
        internationl_cups_button = driver.find_element_by_xpath('/html/body/div[2]/div[6]/div[2]/div[3]').click()

    players_link_international.append(driver.current_url)
    driver.quit()

    
df_links_2 = pd.DataFrame({"Player":total_players,"link_of_player":players_link_international})
df_links_2
df_links_2.to_csv("Links_Of_Players_international.csv")

<h2><span style="text-decoration: underline; color: #000080;">Taking international details of players:</span></h2>
<ul>
<li><span style="color: #000080;">Build big data of all players on CSV</span></li>
<li><span style="color: #000080;">Save all players international details on new data</span></li>
</ul>

In [ ]:

df_international = pd.read_csv("Links_Of_Players_international.csv")
df_international

In [43]:

def Create_DataFrame_champions(url,name):
    url1 = url
    responsel1 = requests.get(url1)
    soup2 = BeautifulSoup(responsel1.content, "html.parser")
    mtag1 = soup2.find("div", attrs = {"class":"table_container"})
    
    player,Season,age,squad,lg_rank,comp,mp,starts,minutes,goals,goals_per_game,assists,
    penalty_kicks_made,penalty_kicks_attempted,
    yellow_cards,red_cards,assists_per_game = [],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]

    j = 0

    for i in mtag1("th", attrs = {"class":"left"}):  # work
        if i.text == '':
            continue
        Season.append(i.text)
    for i in mtag1("td", attrs = {"class":"center"}): # work
        if i.text == '':
            continue
        age.append(i.text)
    for i in mtag1("td", attrs = {"class":"left"}): # work
        if j == 0:
            squad.append(i.text)
            j = j+1
        elif j == 1:
            comp.append(i.text)
            j = j+1
        else:
            j = 0

    j = 0

    for i in mtag1("th", attrs = {"class":"left"}):  # work
        if i.text == '':
            continue
        Season.append(i.text)
    for i in mtag1("td", attrs = {"class":"center"}): # work
        if i.text == '':
            continue
        age.append(i.text)
    for i in mtag1("td", attrs = {"class":"left"}): # work
        if j == 0:
            squad.append(i.text)
            j = j+1
        elif j == 1:
            comp.append(i.text)
            j = j+1
        else:
            j = 0



    for m in mtag1.findAll(id = "stats"):
        j = 0
        for i in m("td", attrs = {"class":"right"}):        
            if(i.text == None):
                continue
            
            if j == 0:
                lg_rank.append(i.text)
                j = j+1
            elif j == 1:
                mp.append(i.text)
                j = j+1    
            elif j == 2:
                starts.append(i.text)
                j = j+1
            elif j == 3:
                minutes.append(i.text)
                j = j+ 1 
            elif j == 4:
                j = j +1
            elif j == 5:
                goals.append(i.text)
                j= j+1
            elif j == 6:
                assists.append(i.text)
                j= j+1
            elif j == 7:
                penalty_kicks_made.append(i.text)
                j= j+1
            elif j == 8:
                penalty_kicks_attempted.append(i.text)
                j= j+1
            elif j == 9:
                yellow_cards.append(i.text)
                j = j+1 
            elif j == 10:
                red_cards.append(i.text)
                j = j+ 1 
            elif j == 11:
                goals_per_game.append(i.text)
                j= j+1
            elif j == 12:
                assists_per_game.append(i.text)
                j= j+1
            elif j < 12:
                break
                
    size = len(goals)
    
    for ijk in squad:
        if ijk == '':
            squad.remove('')
            
    for ijk in squad:
        if ijk == '':
            comp.remove('')
            
            
    while (len(squad) > size):
        squad.pop()

    while (len(Season) > size):
        Season.pop()
    
    while (len(comp) > size):
        comp.pop()    

    while (len(age) > size):
        age.pop()                 

    for names in range (0,size):
        player.append(name)
        
    df_champions = pd.DataFrame({"Player":player,"Season":Season, "Age":age,"Squad":squad,"Lg_Rank":lg_rank,"Comp":comp,"Lg_Rank":lg_rank,"Matches":mp,"Starts":starts,"Minutes":minutes,"Goals":goals,"Assists":assists,"penalty_kicks_made":penalty_kicks_made,"penalty_kicks_attempted":penalty_kicks_attempted,"yellow_cards":yellow_cards,"red_cards":red_cards,"goals_per_game":goals_per_game,"assists_per_game":assists_per_game })         
    return df_champions   

In [ ]:
champions_data = pd.DataFrame()
k = 0
for i in df_international['link_of_player']:
    name = df_international['Player'][k]
    df_assistent = Create_DataFrame_champions(i,name)
    champions_data = pd.concat([champions_data,df_assistent], ignore_index=True, sort=False)
    k = k +1
champions_data  

In [ ]:
champions_data.to_csv("champions_players_data.csv")